In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,21419
2,Huelva,Confirmados PDIA 14 días,5784
3,Huelva,Tasa PDIA 14 días,"1127,1118732583743"
4,Huelva,Confirmados PDIA 7 días,2606
5,Huelva,Total Confirmados,21644
6,Huelva,Curados,11691
7,Huelva,Fallecidos,187


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  21419.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5724.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 225 personas en los últimos 7 días 

Un positivo PCR cada 103 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,21419.0,2606.0,5784.0,513170.0,507.823918,1127.111873,1294.0
Huelva-Costa,13016.0,1577.0,3779.0,289548.0,544.641994,1305.137663,811.0
Condado-Campiña,5859.0,760.0,1496.0,156231.0,486.459153,957.556439,372.0
Huelva (capital),5724.0,640.0,1391.0,143837.0,444.948101,967.066888,351.0
Lepe,1722.0,215.0,488.0,27880.0,771.162123,1750.358680,120.0
Ayamonte,989.0,162.0,445.0,21104.0,767.626990,2108.605004,84.0
Isla Cristina,1783.0,208.0,608.0,21393.0,972.280653,2842.051138,80.0
Sierra de Huelva-Andévalo Central,2194.0,178.0,407.0,67391.0,264.130225,603.938211,61.0
Palma del Condado (La),890.0,93.0,183.0,10801.0,861.031386,1694.287566,56.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,408.0,46.0,130.0,3366.0,1366.607249,3862.150921,17.0
Paymogo,58.0,13.0,38.0,1162.0,1118.760757,3270.223752,4.0
Berrocal,14.0,4.0,9.0,303.0,1320.132013,2970.297030,0.0
Isla Cristina,1783.0,208.0,608.0,21393.0,972.280653,2842.051138,80.0
Villanueva de las Cruces,18.0,2.0,10.0,387.0,516.795866,2583.979328,1.0
San Bartolomé de la Torre,169.0,44.0,92.0,3761.0,1169.901622,2446.157937,23.0
Ayamonte,989.0,162.0,445.0,21104.0,767.626990,2108.605004,84.0
Cabezas Rubias,26.0,9.0,13.0,706.0,1274.787535,1841.359773,5.0
Lepe,1722.0,215.0,488.0,27880.0,771.162123,1750.358680,120.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,197.0,2.0,30.0,4602.0,43.459365,651.890482,0.0,0.066667
Puebla de Guzmán,67.0,3.0,18.0,3092.0,97.024580,582.147477,2.0,0.166667
Higuera de la Sierra,29.0,2.0,11.0,1291.0,154.918668,852.052672,1.0,0.181818
Villanueva de las Cruces,18.0,2.0,10.0,387.0,516.795866,2583.979328,1.0,0.200000
Escacena del Campo,118.0,3.0,13.0,2287.0,131.176213,568.430258,1.0,0.230769
Campillo (El),62.0,1.0,4.0,2024.0,49.407115,197.628458,0.0,0.250000
Encinasola,115.0,6.0,20.0,1310.0,458.015267,1526.717557,0.0,0.300000
Minas de Riotinto,96.0,2.0,6.0,3812.0,52.465897,157.397692,0.0,0.333333
Isla Cristina,1783.0,208.0,608.0,21393.0,972.280653,2842.051138,80.0,0.342105
